
**PROJETO IA - ROTA DE TRÂNSITO IDEAL**





Comandos para instalar as bibliotecas:

-conda install pandas

-conda install haversine

-conda install -c conda-forge folium

**Ou**

-pip install pandas

-pip install folium

-pip install haversine



In [1]:
!pip install pandas

!pip install folium

!pip install haversine

Fazemos as importações necessárias

In [2]:
import requests
import json
import folium
from haversine import haversine
from folium.features import DivIcon
import html.entities
from google.colab import drive

Realizando leitura do JSON

In [3]:
with open('FaroisDoSaberRotas.json') as json_file:
    request_routes = json.load(json_file)

Retornamos 4 rotas possíveis entre cada ponto, agora iremos filtrar entre essas rotas apena as melhores (menos gasto e tempo). Feito isso criamos uma lista com as rotas ideais a serem usadas pela I.A

In [4]:
kml = 14 #COLOCAR GASTO KM/L DO CARRO
routesI = []
for path in request_routes: #Passando por todas as rotas, de todos os pontos
  current_node = path['path_orig']
  decI = 0
  indexRouteI = 0
  for indexRoute, route in enumerate(path['routes_list']):
    distFinal = 0
    routePoints = []
    for index, row in enumerate(route['points']):
        if index < (len(route['points']) - 1):
          pA = (row['latitude'], row['longitude'])
          nextIndex = index + 1
          pB = (route['points'][nextIndex]['latitude'], route['points'][nextIndex]['longitude'])
          distFinal += haversine(pA, pB) #Chamando equação de haversine, calcula distância entre duas coordenadas
        routePoints.append([row['latitude'], row['longitude']])
    gasto = round(distFinal * kml, 0)
    pesoDec = (gasto * route['travelTime']) #Multiplicando os dois fatores para ter um peso de decisão
    if decI == 0 or decI > pesoDec:
      decI = pesoDec #Se estiver entre as rotas ideais, salva
      pointsI = routePoints 
  edge = {'dest_node':path['path_dest'], 'edge_cost': decI, 'route_points':pointsI}
  found_node = False
  for routeIndex, routeI in enumerate(routesI): #Passando pela lista de rotas ideais
    if routeI['orig_node'] == current_node: #Se achar um item na lista cujo nó de origem é igual a origem atual, usa ele
      routesI[routeIndex]['dest'].append(edge) #Coloca o destino
      found_node = True
  if not found_node: #Senão encontrar o nó origem atual (ponto) na lista de rotas ideias, cria ele e coloca um ponto de destino
    new_routeI = {'orig_node':current_node, 'dest':[]}
    new_routeI['dest'].append(edge)
    routesI.append(new_routeI)

Busca pelo melhor caminho
---



In [5]:
idealRoutes = routesI
currentPoint = 0
arrivalPoint = 41
lowerValue = 0
iterations = 0
lastPoints = [currentPoint]
path = ''
isPassed = False
optRoute = []

def bestPath(currentPoint2):
  global idealRoutes
  global currentPoint
  global arrivalPoint
  global lowerValue
  global iterations
  global lastPoints
  global path
  global isPassed

  vetor = idealRoutes[currentPoint2]['dest']

  for value in vetor:
      if lowerValue == 0 or lowerValue > value['edge_cost']:
        
        for item in lastPoints: #Valida se já passou pelo ponto antes
            if item == value['dest_node']:
              isPassed = True

        if value['dest_node'] == arrivalPoint and len(lastPoints) != len(idealRoutes): 
            continue
        
        if len(lastPoints) == len(idealRoutes): #Se pontos passados for igual número de pontos, estamos no fim da rota
            currentPoint = arrivalPoint
        
        if isPassed == False: #Se não se passou por esse ponto ainda
            lowerValue = value['edge_cost'];
            currentPoint = value['dest_node'];

      isPassed = False

  optRoute.append(currentPoint)
 
  lastPoints.append(currentPoint)

  if iterations >= len(idealRoutes) or currentPoint == arrivalPoint: # Segunda condição apenas para não ocorrer erros.
      return
  else:
      iterations = iterations + 1
      lowerValue = 0
      bestPath(currentPoint)

bestPath(currentPoint)



Criamos uma função que criará uma lista de coordenadas na ordem da rota ideal

In [6]:
def createCoordsRoute(optRoute):
  coordsRoute = []
  currentNode = 0
  for idx, route in enumerate(optRoute):
    for edge in routesI:
      if edge['orig_node'] == currentNode:
          nextNode = optRoute[idx]
          for destNode in edge['dest']:
            if destNode['dest_node'] == nextNode:
              for coords in destNode['route_points']:
                coordsRoute.append(coords)
                currentNode = nextNode
          break
  return coordsRoute

Criamos uma lista com todas as coordenadas da rota, na ordem escolhida pela busca gulosa

In [7]:
coordsRoute = createCoordsRoute(optRoute)

Carregando Arquivos JSON's com nomes e coordenadas dos endereços

In [8]:
with open('enderecosFarol.json') as json_file:
    coordsEnde = json.load(json_file)

Cria-se uma tabela contendo os caracteres especiais em HTML e seus correspondentes em ASCII, será usado na exibição

In [9]:
global tableCharacters 
tableCharacters = {k: '&{};'.format(v) for k, v in html.entities.codepoint2name.items()}

Definindo a função para criação do mapa

In [11]:
def CreateMap(coordsEnde, coordsRoute):
  m = folium.Map(location=[float(coordsEnde[0]['coord'][1]), float(coordsEnde[0]['coord'][0])],
               zoom_start=13) #Criamos um objeto mapa da biblioteca Folium, será usado para exibição

  folium.Marker([coordsEnde[0]['coord'][1], coordsEnde[0]['coord'][0]], 
              popup='<b>' + coordsEnde[0]['NomeLugar'] + '</b>', 
              icon=DivIcon(icon_size=(200,36),
                icon_anchor=(0,0),
                html='<div style="font-size: 12pt; color:blue; font-weight: bold">Ponto Inicial</div>' #Criamos uma legenda indicando o ponto inicial
              )).add_to(m)

  for index, ende in enumerate(coordsEnde): #Criamos um ponto no mapa para cada parada
    folium.Marker([ende['coord'][1], ende['coord'][0]], 
                  popup='<b>' + ende['NomeLugar'].translate(tableCharacters) + '</b>',  
                  icon = folium.features.CustomIcon('Marker.png', icon_size=(18,18))).add_to(m)

  tuples = [tuple(x) for x in coordsRoute] #Percorremos a rota do parâmetro criando uma linha verde indicando o caminho
  folium.PolyLine(tuples, color="green", weight=2.5, opacity=1).add_to(m) 

  return m #Retornamos o mapa


Criamos o objeto para mostrar o mapa da rota criada

In [12]:
m = CreateMap(coordsEnde, coordsRoute)

Exibimos o mapa

In [ ]:
m

Agora tentaremos otimizar a rota, a função abaixo serve para identificar o custo total da rota, multiplicando distância por tempo

In [15]:
def CostRoute(request_routes, optRoute):
  current_node = 0
  next_node = optRoute[0]
  index_route = 0
  totalTime = 0
  distFinal = 0
  nodes = 0
  for node in optRoute:
    for path in request_routes:
      if path['path_orig'] == current_node and path['path_dest'] == node:
        nodes += 1
        totalTime += path['routes_list'][0]['travelTime']
        for index, row in enumerate(path['routes_list'][0]['points']):
          if index < (len(path['routes_list'][0]['points']) - 1):
            pA = (row['latitude'], row['longitude'])
            nextIndex = index + 1
            pB = (path['routes_list'][0]['points'][nextIndex]['latitude'], path['routes_list'][0]['points'][nextIndex]['longitude'])
            distFinal += haversine(pA, pB) #Chamando equação de haversine, calcula distância entre duas coordenadas
        current_node = node
        break 
  return (distFinal * totalTime)

Utilizaremos 2-opt para realizar a otimização, abaixo a definição da função 2OptSwap

In [16]:
def TwoOptSwap(route, i, k): 
#A função recebe uma lista e dois pontos, a ideia é inverter o trecho da lista 
#entre esses dois pontos do parâmetro e manter o resto da lista como antes
  new_route = []
  for routePoint in route[0:i]: #Até o ponto I, a nova rota é idêntica a original
    new_route.append(routePoint)
  for routePoint in reversed(route[i:k + 1]): #Do ponto I ao K ela retorna invertida
    new_route.append(routePoint)
  for routePoint in route[k+1:(len(route))]: #Do ponto K em diante mantém-se idêntica
    new_route.append(routePoint)
  return new_route #Retornamos a nova rota

Criamos a função de implementação, passaremos por todos os pontos da rota e tentaremos as inversões até obter a melhor rota

In [17]:
def TwoOptImplement(route):
  best_cost = CostRoute(request_routes, route)
  for i in range(1, (len(route) - 2)):
    for k in range((i + 1), (len(route) - 2)):
      new_route = TwoOptSwap(route, i, k)
      new_cost = CostRoute(request_routes, new_route)
      if (new_cost < best_cost):
        route = new_route
        best_distance = new_cost
  return route

Realizamos a aplicação do 2-opt

In [18]:
new_optRoute = TwoOptImplement(optRoute)

Executamos a função de custo para verificar se tivemos ganhos

In [19]:
print(CostRoute(request_routes, optRoute))
print(CostRoute(request_routes, new_optRoute))

3444985.896215819
3374683.8728551273


Criamos uma nova lista de coordenadas

In [20]:
optCoordsRoute = createCoordsRoute(new_optRoute)

Criamos o novo mapa

In [21]:
m_new = CreateMap(coordsEnde, optCoordsRoute)

E exibimos

In [ ]:
m_new